In [3]:
import numpy as np
import pandas as pd
import os
import sys
import sklearn

In [4]:
main_dir = '/Users/ishitamed/Downloads/Predict-Influencer-Dataset'

In [5]:
test_data = pd.read_csv(os.path.join(main_dir,'test.csv'))
train_data = pd.read_csv(os.path.join(main_dir,'train.csv'))

In [6]:
train_data.head()

,Choice,A_follower_count,A_following_count,A_listed_count,A_mentions_received,A_retweets_received,A_mentions_sent,A_retweets_sent,A_posts,A_network_feature_1,...,B_following_count,B_listed_count,B_mentions_received,B_retweets_received,B_mentions_sent,B_retweets_sent,B_posts,B_network_feature_1,B_network_feature_2,B_network_feature_3
0,0,228,302,3,0.583979,0.100503,0.100503,0.100503,0.362150,2,...,29808,1689,15.430498,3.984029,8.204331,0.332423,6.988815,66,75.530303,1916.893939
1,0,21591,1179,228,90.456506,25.798292,5.709329,1.111159,5.176620,369,...,848,1610,40.495021,8.943607,3.227677,0.564343,1.070321,163,132.030675,2931.515337
2,0,7310,1215,101,25.503644,9.556347,5.361519,0.591206,3.589718,95,...,482,206,0.734696,0.354379,0.603202,0.100503,0.603202,3,10.333333,277.333333
3,0,20,7,2,7.690824,0.277306,1.331508,0.100503,2.830627,6,...,17637,278,572.874856,390.293681,27.552040,7.167557,32.101906,1762,23.469296,1395.845634
4,1,45589,862,2641,148.854279,36.998884,27.881768,3.333492,23.861282,551,...,1711,181,21.601866,3.581661,6.764657,1.119727,4.563246,85,48.500000,1993.627907


In [7]:
print(train_data.describe())

            Choice  A_follower_count  A_following_count  A_listed_count  \
count  5500.000000      5.500000e+03       5.500000e+03     5500.000000   
mean      0.509455      6.498840e+05       1.265895e+04     5952.453273   
std       0.499956      2.028787e+06       4.900867e+04    17339.141191   
min       0.000000      1.600000e+01       0.000000e+00        0.000000   
25%       0.000000      2.663750e+03       3.220000e+02       85.000000   
50%       1.000000      4.558900e+04       7.780000e+02      932.000000   
75%       1.000000      3.927380e+05       2.838000e+03     6734.000000   
max       1.000000      3.654319e+07       1.165830e+06   549144.000000   

       A_mentions_received  A_retweets_received  A_mentions_sent  \
count         5.500000e+03          5500.000000      5500.000000   
mean          2.666032e+03          1032.371839         6.011873   
std           2.916543e+04         10954.953223         9.519797   
min           1.005034e-01             0.100503     

In [8]:
def pre_pro(df):
    df = df.astype('float32')
    col = df.columns
    for i in range(len(col)):
        m = df.loc[df[col[i]] != -np.inf, col[i]].min()
        df[col[i]].replace(-np.inf,m,inplace=True)
        M = df.loc[df[col[i]] != np.inf, col[i]].max()
        df[col[i]].replace(np.inf,M,inplace=True)
    
    df.fillna(0, inplace = True)
    return df 

In [9]:
def feat_eng(df):
    df.replace(0, 0.001)
    
    df['follower_diff'] = (df['A_follower_count'] > df['B_follower_count'])
    df['following_diff'] = (df['A_following_count'] > df['B_following_count'])
    df['listed_diff'] = (df['A_listed_count'] > df['B_listed_count'])
    df['ment_rec_diff'] = (df['A_mentions_received'] > df['B_mentions_received'])
    df['rt_rec_diff'] = (df['A_retweets_received'] > df['B_retweets_received'])
    df['ment_sent_diff'] = (df['A_mentions_sent'] > df['B_mentions_sent'])
    df['rt_sent_diff'] = (df['A_retweets_sent'] > df['B_retweets_sent'])
    df['posts_diff'] = (df['A_posts'] > df['B_posts'])

    df['A_pop_ratio'] = df['A_mentions_sent']/df['A_listed_count']
    df['A_foll_ratio'] = df['A_follower_count']/df['A_following_count']
    df['A_ment_ratio'] = df['A_mentions_sent']/df['A_mentions_received']
    df['A_rt_ratio'] = df['A_retweets_sent']/df['A_retweets_received']
    
    df['B_pop_ratio'] = df['B_mentions_sent']/df['B_listed_count']
    df['B_foll_ratio'] = df['B_follower_count']/df['B_following_count']
    df['B_ment_ratio'] = df['B_mentions_sent']/df['B_mentions_received']
    df['B_rt_ratio'] = df['B_retweets_sent']/df['B_retweets_received']
    
    df['A/B_foll_ratio'] = (df['A_foll_ratio'] > df['B_foll_ratio'])
    df['A/B_ment_ratio'] = (df['A_ment_ratio'] > df['B_ment_ratio'])
    df['A/B_rt_ratio'] = (df['A_rt_ratio'] > df['B_rt_ratio'])

    df['nf1_diff'] = (df['A_network_feature_1'] > df['B_network_feature_1'])
    df['nf2_diff'] = (df['A_network_feature_2'] > df['B_network_feature_2'])
    df['nf3_diff'] = (df['A_network_feature_3'] > df['B_network_feature_3'])
    
    df['nf3_ratio'] = df['A_network_feature_3'] / df['B_network_feature_3']
    df['nf2_ratio'] = df['A_network_feature_2'] / df['B_network_feature_2']
    df['nf1_ratio'] = df['A_network_feature_1'] / df['B_network_feature_1']
    
    return(pre_pro(df))

In [10]:
proc_train = feat_eng(train_data.copy())
proc_test = feat_eng(test_data.copy())

In [11]:
X = proc_train.iloc[:,1:].as_matrix()
Y = np.reshape(proc_train["Choice"].as_matrix(), (-1, 1))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [12]:

print("X: ", X.shape)

print("Y: ",Y.shape)

X:  (5500, 47)
Y:  (5500, 1)


# Simple Nueral Network

Public Score - 0.8200

In [41]:
from sklearn.model_selection import train_test_split
X_train, X_validate, Y_train, Y_validate = sklearn.model_selection.train_test_split(X, Y, test_size=0.25, random_state=42)

In [42]:
print("Training set size: ", X_train.shape)
print("Validation set size: ", X_validate.shape)

Training set size:  (4125, 47)
Validation set size:  (1375, 47)


In [45]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from sklearn.metrics import roc_curve

Using TensorFlow backend.


ModuleNotFoundError: No module named 'callback'

In [46]:
model = Sequential()

model.add(Dense(50, activation='relu', input_dim=47, kernel_initializer='uniform'))
model.add(Dropout(0.5))
model.add(Dense(10, kernel_initializer='uniform'))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid', kernel_initializer='uniform'))

model.compile(loss='binary_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])

W1217 20:23:50.400213 4702059968 deprecation.py:323] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [47]:
model.fit(X_train, Y_train, epochs=20, batch_size=64, validation_data=(X_validate, Y_validate))


W1217 20:24:28.681454 4702059968 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Train on 4125 samples, validate on 1375 samples
Epoch 1/20
4125/4125 [==============================] - 1s 127us/step - loss: 0.6687 - accuracy: 0.6330 - val_loss: 0.6680 - val_accuracy: 0.7062
Epoch 2/20
4125/4125 [==============================] - 0s 61us/step - loss: 0.6471 - accuracy: 0.6778 - val_loss: 0.6727 - val_accuracy: 0.7011
Epoch 3/20
4125/4125 [==============================] - 0s 54us/step - loss: 0.6379 - accuracy: 0.6696 - val_loss: 0.6538 - val_accuracy: 0.7324
Epoch 4/20
4125/4125 [==============================] - 0s 33us/step - loss: 0.6260 - accuracy: 0.6754 - val_loss: 0.6381 - val_accuracy: 0.7484
Epoch 5/20
4125/4125 [==============================] - 0s 32us/step - loss: 0.6162 - accuracy: 0.6975 - val_loss: 0.6280 - val_accuracy: 0.7462
Epoch 6/20
4125/4125 [==============================] - 0s 33us/step - loss: 0.5999 - accuracy: 0.6989 - val_loss: 0.6156 - val_accuracy: 0.7455
Epoch 7/20
4125/4125 [==============================] - 0s 47us/step - loss: 0.58

In [51]:
X_test = proc_test.as_matrix()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [52]:
print(X_test.shape)

(5952, 47)


In [53]:
Y_pred = model.predict(X_test)

In [54]:
sample_submission = pd.read_csv(os.path.join(main_dir,'sample_predictions.csv'))

In [55]:
sample_submission["Choice"] = Y_pred

In [57]:
sample_submission.to_csv('./submission_nn.csv', index=False)

# Random Forest

Public Score - 0.82

In [58]:
from sklearn.ensemble import RandomForestClassifier

# Create the model with 100 trees
model_rf = RandomForestClassifier(n_estimators=175, 
                               bootstrap = True,
                               max_features = 'sqrt')
# Fit on training data
model_rf.fit(X_train, Y_train)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=175,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [84]:
# Actual class predictions
y_pred = model.predict(X_validate)
print(y_pred)

[[0.37838826]
 [0.5604128 ]
 [0.83372647]
 ...
 [0.08401162]
 [0.85313624]
 [0.33359203]]


In [85]:
Y_pred = []
Y_test = []
for val in y_pred:
    if val>0.5:
        Y_pred.append(1)
    else:
        Y_pred.append(0)

In [86]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(Y_validate, Y_pred))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(Y_validate, Y_pred))

# Model Recall: what percentage of positive tuples are labelled as such?
#print("Recall:",metrics.recall_score(Y_validate, np.asarray(Y_pred)))

# print("F1 score:",metrics.f1_score(Y_validate, np.asarray(Y_pred)))

Accuracy: 0.7556363636363637
Precision: 0.7556818181818182


In [87]:
# Actual class predictions
y_test = model.predict(X_test)
print(y_test)

[[0.49160048]
 [0.4957737 ]
 [0.38750127]
 ...
 [0.50326276]
 [0.49542114]
 [0.49012652]]


In [89]:
sample_submission["Choice"] = y_test

In [90]:
sample_submission.to_csv('./submission_rf.csv', index=False)